In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Optional, List, Callable
import json
import pickle

import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from torch.optim.lr_scheduler import OneCycleLR

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint


print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")

# local modules
import sys
sys.path.append("../src")
from preproc import process_train_data, process_test_data
from transformer import LGBMLeavesEncoder

In [2]:
from pytorch_tabular.models.common.layers import GatedFeatureLearningUnit
from pytorch_tabular.models.common.layers.activations import t_softmax

In [ ]:
# set seeds for reproducibility
np.random.seed(2112)
pl.seed_everything(2112)

In [4]:
# useful callbacks
class LearningRateMonitor(pl.Callback):
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        if batch_idx % 100 == 0:  # Log every 100 batches
            lr = pl_module.optimizers().param_groups[0]['lr']
            pl_module.log('learning_rate', lr, prog_bar=True)   

class BestValRMSELogger(pl.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_rmse = float('inf')

    def on_validation_epoch_end(self, trainer, pl_module):
        current_val_rmse = trainer.callback_metrics.get('val_rmse')
        if current_val_rmse is not None:
            self.best_val_rmse = min(self.best_val_rmse, current_val_rmse)
            pl_module.log('best_val_rmse', self.best_val_rmse, prog_bar=True)

***
### load and preprocess data

In [ ]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

In [6]:
# Load the feature selection results
fs_type = 'uni95'

In [ ]:
fs_mapping = {
    "full": None,
    "fsv2": '../feat_selection/select_optuna_lgbm_v2.json',
    "fsv3": '../feat_selection/select_optuna_catb.json',
    "fsv4": '../feat_selection/select_optuna_lgbm_v3.json',
    "fsv23": '../feat_selection/select_optuna_combined_v23.json',
    "fsv24": '../feat_selection/select_optuna_combined_v24.json',
    "fsv34": '../feat_selection/select_optuna_combined_v34.json',
    "int95": '../feat_selection/feat_selection_intersection_at_95.json',
    "int96": '../feat_selection/feat_selection_intersection_at_96.json',
    "int97": '../feat_selection/feat_selection_intersection_at_97.json',
    "int98": '../feat_selection/feat_selection_intersection_at_98.json',
    "int99": '../feat_selection/feat_selection_intersection_at_99.json',
    "uni80": '../feat_selection/feat_selection_union_at_80.json',
    "uni85": '../feat_selection/feat_selection_union_at_85.json',
    "uni90": '../feat_selection/feat_selection_union_at_90.json',
    "uni95": '../feat_selection/feat_selection_union_at_95.json',
}

fs_path = fs_mapping[fs_type]

if fs_path is None:
    feature_selection = dict()
else:
    with open(fs_path, 'r') as f:
        feature_selection = json.load(f)

# Extract the selected features
numerical_cols = feature_selection.get('numerical', None)
categorical_cols = feature_selection.get('categorical', None)

# text_cols = ["LudRules",]
text_cols = list()

print("Numerical features:", len(numerical_cols) if numerical_cols else 0)
print("Categorical features:", len(categorical_cols) if categorical_cols else 0)
print("Text features:", len(text_cols) if text_cols else 0)

In [ ]:
df_train, numerical_cols, categorical_cols, encoder, scaler = process_train_data(
    df_train,
    scale=True,
    scale_type='minmax',
    numerical_cols=numerical_cols,
    categorical_cols=categorical_cols,
    include_position_features=False,
    include_text_features=False,
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

In [ ]:
plt.hist(df_train['utility_agent1'], bins=100)
plt.title('Distribution of Target Variable')
plt.xlabel('Utility Agent 1')
plt.ylabel('Frequency')
plt.show()

In [ ]:
cat_input_dims = df_train[categorical_cols].nunique(axis=0).values.tolist()
print(cat_input_dims)

In [11]:
split1 = pickle.load(open('../data/splits/cv1_Game.pkl', 'rb'))
split2 = pickle.load(open('../data/splits/cv2_Game.pkl', 'rb'))
split3 = pickle.load(open('../data/splits/cv3_Game.pkl', 'rb'))

# split1 = pickle.load(open('../data/splits/cv1_GameRulesetName.pkl', 'rb'))
# split2 = pickle.load(open('../data/splits/cv2_GameRulesetName.pkl', 'rb'))
# split3 = pickle.load(open('../data/splits/cv3_GameRulesetName.pkl', 'rb'))

***
### train model


In [12]:
class GANDALF(pl.LightningModule):

    def __init__(self, 
            num_input_dim: int,
            cat_input_dims: list[int],
            output_dim: int,
            dropout: float,
            embedding_dropout: float,
            learning_rate: float = 1e-3,
            weight_decay: float = 1e-5,
            initialization: str = 'kaiming_uniform',
            embedding_dim: Optional[List[int]] = None,
            pct_start: float = 0.2,
            div_factor: float = 10.0,
            final_div_factor: float = 1e4,
            n_stages: int = 6,
            feature_mask_function: Callable = t_softmax,
            feature_sparsity: float = 0.3,
            learnable_sparsity: bool = True
        ):
        super().__init__()
        self.save_hyperparameters()
        self.dropout = dropout
        self.embedding_dropout = embedding_dropout
        self.pct_start = pct_start
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor

        # Initialize embedding dimensions if not provided
        if embedding_dim is None:
            # Rule of thumb: min(50, num_unique // 2 + 1) for each categorical feature
            embedding_dim = [min(50, int(1 + np.ceil(np.sqrt(dim)))) for dim in cat_input_dims]

        elif len(embedding_dim) != len(cat_input_dims):
            raise ValueError("Length of embedding_dim must match number of categorical features.")

        self.embedding_dim = embedding_dim

        # Create embedding layers
        self.create_embeddings(cat_input_dims, embedding_dim)

        # Create backbone layers
        self.create_backbone(
            num_input_dim,
            n_stages,
            feature_mask_function,
            dropout,
            feature_sparsity,
            learnable_sparsity
        )

        # Create head layers
        self.create_head(output_dim)

        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.initialization = initialization

        self._init_weights()

        # Initialize lists to store validation outputs
        self.validation_targets = []
        self.validation_predictions = []

    def create_embeddings(self, cat_input_dims: list[int], embedding_dim: list[int]):
        self.embeddings = nn.ModuleList(
            [nn.Embedding(dim, emb_dim) for dim, emb_dim in zip(cat_input_dims, embedding_dim)]
        )
        self.embedding_dropout_layer = nn.Dropout(self.embedding_dropout)

    def create_backbone(
            self, 
            num_input_dim: int,
            n_stages: int,
            feature_mask_function: Callable,
            dropout: float,
            feature_sparsity: float,
            learnable_sparsity: bool
        ):
        # Calculate total input dimension after embeddings
        total_embedding_dim = sum(self.embedding_dim)
        total_input_dim = num_input_dim + total_embedding_dim

        self.backbone = GatedFeatureLearningUnit(
            n_features_in=total_input_dim,
            n_stages=n_stages,
            feature_mask_function=feature_mask_function,
            dropout=dropout,
            feature_sparsity=feature_sparsity,
            learnable_sparsity=learnable_sparsity,
        )
        self.backbone_output_size = total_input_dim

    def create_head(self, output_dim: int):
        # Output layer
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.backbone_output_size),
            nn.Linear(self.backbone_output_size, output_dim)
        )

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                if any(module is m for m in self.head.modules()):
                    nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('tanh'))
                else:
                    if self.initialization == 'kaiming_uniform':
                        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'kaiming_normal':
                        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'xavier_uniform':
                        nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
                    elif self.initialization == 'xavier_normal':
                        nn.init.xavier_normal_(module.weight, gain=nn.init.calculate_gain('relu'))
                    else:
                        raise ValueError(f"Unsupported initialization method: {self.initialization}")
                
                # Initialize bias to small values
                if module.bias is not None:
                    nn.init.uniform_(module.bias, -0.1, 0.1)

    def forward(self, x_num, x_cat):
        # Process categorical variables
        embedded = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        embedded = self.embedding_dropout_layer(embedded)
        
        # Concatenate numerical and embedded categorical features
        x = torch.cat([x_num, embedded], dim=1)
        
        # Pass through backbone
        x = self.backbone(x)
        
        # Pass through head
        x = self.head(x)
        x = nn.functional.hardtanh(x)

        return x.squeeze(-1)

    def training_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('valid_loss', loss, prog_bar=True)
        # Store targets and predictions for later use
        self.validation_targets.append(y)
        self.validation_predictions.append(y_hat)
        return loss
    
    def predict_step(self, batch, batch_idx):
        if len(batch) == 2:
            x_num, x_cat = batch
        elif len(batch) == 3:
            x_num, x_cat, _ = batch
        y_hat = self(x_num, x_cat)
        return y_hat

    def on_validation_epoch_end(self):
        # Concatenate all targets and predictions
        y = torch.cat(self.validation_targets)
        y_hat = torch.cat(self.validation_predictions)
        rmse = torch.sqrt(F.mse_loss(y_hat, y))
        self.log('val_rmse', rmse, prog_bar=True)
        # Clear the lists for next epoch
        self.validation_targets.clear()
        self.validation_predictions.clear()
                
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.learning_rate, 
            weight_decay=self.weight_decay,
        )
        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.learning_rate,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=self.pct_start,
            div_factor=self.div_factor,
            final_div_factor=self.final_div_factor,
            anneal_strategy='cos',
            cycle_momentum=True,
            base_momentum=0.85,
            max_momentum=0.95,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [13]:
params = {
    'batch_size': 512,
    'embedding_dropout': 0.2,
    
    # GANDALF config
    'dropout': 0.0,
    'feature_sparsity': 0.8,
    'learnable_sparsity': True,
    'n_stages': 1,

    'learning_rate': 5e-3,
    'weight_decay': 1e-5,
    'pct_start': 0.05,
    'div_factor': 100.0,
    'final_div_factor': 1.
}

In [14]:
def train_models(
        df_train,
        numerical_cols,
        categorical_cols,
        target='utility_agent1',
        split_list=None,
    ):
    """Train multiple GANDALF models using cross-validation.
    
    Args:
        df_train: Training dataframe
        numerical_cols: List of numerical column names
        categorical_cols: List of categorical column names 
        target: Target column name
        split_list: List of train/val splits for cross-validation
    
    Returns:
        trained_models: List of trained GANDALF models
        lgbm_encoders: List of fitted LGBM encoders
        oof: Out-of-fold predictions dataframe
        oof_scores: List of RMSE scores for each fold
    """
    trained_models = list()
    lgbm_encoders = list()
    oof = pd.DataFrame(index=df_train.index, columns=[f'{target}_true', f'{target}_pred'])
    oof_scores = []

    # Perform cross-validation
    for fold, (train_index, val_index) in enumerate(split_list, 1):
        print(f"Fold {fold}")
        
        # Split the data
        X_train = df_train.iloc[train_index][numerical_cols + categorical_cols]
        y_train = df_train.iloc[train_index][target]
        X_valid = df_train.iloc[val_index][numerical_cols + categorical_cols]
        y_valid = df_train.iloc[val_index][target]

        lgbm_encoder = LGBMLeavesEncoder(
            num_cols=numerical_cols,
            cat_cols=categorical_cols,
            task='regression',
            n_estimators=100,
            num_leaves=127,
            random_state=2112,
            verbosity=-1,
        )
        lgbm_encoder.fit(X_train, y_train)
        X_train_leaves_lgbm = lgbm_encoder.transform(X_train, verbose=True)
        X_valid_leaves_lgbm = lgbm_encoder.transform(X_valid)
        
        X_train = pd.concat([X_train, X_train_leaves_lgbm], axis=1)
        X_valid = pd.concat([X_valid, X_valid_leaves_lgbm], axis=1) 

        _categorical_cols = (
            categorical_cols + lgbm_encoder.new_columns
        )
        _cat_input_dims = (
            df_train[categorical_cols].nunique(axis=0).values.tolist() + [127]*len(lgbm_encoder.new_columns)
        )

        lgbm_encoders.append(lgbm_encoder)

        train_dataset = TensorDataset(
            torch.tensor(X_train[numerical_cols].values, dtype=torch.float32),
            torch.tensor(X_train[_categorical_cols].values, dtype=torch.int32),
            torch.tensor(y_train.values, dtype=torch.float32)
        )
        train_loader = DataLoader(
            train_dataset,
            batch_size=params['batch_size'],
            shuffle=True,
            num_workers=8,
            persistent_workers=True
        )

        valid_dataset = TensorDataset(
            torch.tensor(X_valid[numerical_cols].values, dtype=torch.float32),
            torch.tensor(X_valid[_categorical_cols].values, dtype=torch.int32),
            torch.tensor(y_valid.values, dtype=torch.float32)
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=params['batch_size'],
            shuffle=False,
            num_workers=8,
            persistent_workers=True
        )

        model = GANDALF(
            num_input_dim=len(numerical_cols),
            cat_input_dims=_cat_input_dims,
            output_dim=1,
            embedding_dropout=params['embedding_dropout'],
            learning_rate=params['learning_rate'], 
            weight_decay=params['weight_decay'],
            initialization="kaiming_uniform",
            pct_start=params["pct_start"],
            div_factor=params["div_factor"],
            final_div_factor=params["final_div_factor"],
            # GANDALF config
            dropout=params['dropout'],
            n_stages=params['n_stages'],
            feature_sparsity=params['feature_sparsity'],
            learnable_sparsity=params['learnable_sparsity'],
        )
        trainer = pl.Trainer(
            max_epochs=100,
            accelerator="mps", 
            callbacks=[
                EarlyStopping(
                    monitor='val_rmse',
                    patience=10,
                    mode='min',
                    verbose=False
                ), 
                LearningRateMonitor(), 
                BestValRMSELogger(),
                ModelCheckpoint(monitor='val_rmse', mode='min', save_top_k=1),
            ],
        )
        trainer.fit(
            model, 
            train_loader,
            valid_loader,
        )

        # Load the best model
        best_model_path = trainer.checkpoint_callback.best_model_path
        model = GANDALF.load_from_checkpoint(best_model_path)
        trained_models.append(model)

        # Predict on validation set using trainer.predict with the prediction DataLoader
        predictions = trainer.predict(model, dataloaders=valid_loader)
        y_pred = torch.cat(predictions).squeeze().cpu().numpy()
         
        # Compute RMSE on scaled values
        y_valid = y_valid.values
        rmse = np.sqrt(np.mean((y_pred - y_valid) ** 2))
        print(f"Fold {fold} - RMSE: {rmse}")

        # Save out-of-fold predictions
        oof.loc[val_index, f'{target}_true'] = y_valid
        oof.loc[val_index, f'{target}_pred'] = y_pred
        oof.loc[val_index, 'fold'] = fold

        # Save RMSE to the list
        oof_scores.append(rmse)

    # Print the list of oof scores and average oof score
    print("List of oof scores:", oof_scores)
    print("Average oof score:", np.mean(oof_scores))

    return trained_models, lgbm_encoders, oof, oof_scores

In [ ]:
# call the function
trained_models1, lgbm_encoders1, oof1, oof_scores1 = train_models(
    df_train, 
    numerical_cols, 
    categorical_cols, 
    target='utility_agent1',
    split_list=split1,
)
# save oof predictions
oof1.to_parquet(path_results / f'oof_nn-gandalf_{fs_type}_cv1.parquet', index=False)

In [ ]:
# call the function
trained_models2, lgbm_encoders2, oof2, oof_scores2 = train_models(
    df_train, 
    numerical_cols, 
    categorical_cols, 
    target='utility_agent1',
    split_list=split2,
)
# save oof predictions
oof2.to_parquet(path_results / f'oof_nn-gandalf_{fs_type}_cv2.parquet', index=False)

In [ ]:
# call the function
trained_models3, lgbm_encoders3, oof3, oof_scores3 = train_models(
    df_train, 
    numerical_cols, 
    categorical_cols, 
    target='utility_agent1',
    split_list=split3,
)
# save oof predictions
oof3.to_parquet(path_results / f'oof_nn-gandalf_{fs_type}_cv3.parquet', index=False)

In [ ]:
trained_models = [*trained_models1, *trained_models2, *trained_models3]
lgbm_encoders = [*lgbm_encoders1, *lgbm_encoders2, *lgbm_encoders3]

In [ ]:
# Print summary of OOF scores for each CV fold
print("CV1 OOF Scores:")
for score in oof_scores1:
    print(f"Score: {score:.4f}")
print(f"Average CV1 Score: {sum(oof_scores1)/len(oof_scores1):.4f}")
    
print("\nCV2 OOF Scores:")  
for score in oof_scores2:
    print(f"Score: {score:.4f}")
print(f"Average CV2 Score: {sum(oof_scores2)/len(oof_scores2):.4f}")
    
print("\nCV3 OOF Scores:")
for score in oof_scores3:
    print(f"Score: {score:.4f}")

In [ ]:
plt.hist(oof1['utility_agent1_true'], alpha=0.5, label='Target')
plt.hist(oof1['utility_agent1_pred'], alpha=0.5, label='Prediction')
plt.legend()
plt.show()

***
### inference

In [11]:
import os
import polars as pl
sys.path.append("../data/raw/")
import kaggle_evaluation.mcts_inference_server

In [12]:
def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    # Convert Polars DataFrame to Pandas DataFrame
    test_pd = test.to_pandas()
    
    # Process the test data
    test_processed = process_test_data(
        test_pd,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler
    )
        
    # Initialize an array to store predictions
    predictions = np.zeros(len(test_pd))
    
    # Make predictions using each trained model
    for lgbm_encoder,model in zip(lgbm_encoders, trained_models):

        # Select relevant features
        X_test_num = test_processed[numerical_cols].copy()
        X_test_cat = test_processed[categorical_cols].copy()

        # Add LGBM encoder leaves features
        lgbm_features = lgbm_encoder.transform(
            test_processed[numerical_cols+categorical_cols]
        )
        X_test_cat = pd.concat([X_test_cat, lgbm_features], axis=1)
        _categorical_cols = categorical_cols + lgbm_encoder.new_columns

        model.eval()
        with torch.no_grad():
            X_test_num_tensor = torch.tensor(
                X_test_num[numerical_cols].values, dtype=torch.float32
            )
            X_test_cat_tensor = torch.tensor(
                X_test_cat[_categorical_cols].values, dtype=torch.int32
            )
            batch_predictions = model(
                X_test_num_tensor, X_test_cat_tensor
            ).cpu().numpy().flatten()

        predictions += batch_predictions
    
    # Average the predictions
    predictions /= len(trained_models)
    
    # Create the submission DataFrame
    submission = sample_sub.with_columns(pl.Series("utility_agent1", predictions))
    
    return submission

In [ ]:
test = pl.read_csv('../data/raw/test.csv')
sample_sub = pl.read_csv('../data/raw/sample_submission.csv')
predict(test, sample_sub)

In [13]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '../data/raw/test.csv',
            '../data/raw/sample_submission.csv'
        )
    )

In [ ]:
torch_save = {
    'models': [model.state_dict() for model in trained_models],
    'models_hparams': [model.hparams for model in trained_models],
    'numerical_cols': numerical_cols,
    'categorical_cols': categorical_cols,
    'encoder': encoder,
    'scaler': scaler,
    'lgbm_encoders': lgbm_encoders
}
torch.save(torch_save, f'./nn-gandalf_predict_{fs_type}.pt')

***